In [474]:
import numpy as np
from random import randint
import math
import unittest
import operator
import numbers
import sys
import random

In [475]:
class TicTacToe:

    # There are 5812 legal board states that can be reached before there is a winner
    # http://brianshourd.com/posts/2012-11-06-tilt-number-of-tic-tac-toe-boards.html
    
    __bad_move_game_is_over = -1
    __bad_move_action_already_played = -2
    __bad_move_no_consecutive_plays = -3
    __play = float(1) # reward for playing an action
    __draw = float(10) # reward for playing to end but no one wins
    __win = float(5) # reward for winning a game
    __loss = float(-10) # reward (penalty) for losing a game
    __rewards = {"Play":TicTacToe.__play,"Draw":TicTacToe.__draw,"Win":TicTacToe.__win,"Loss":TicTacToe.__loss}
    __no_player = -2 # id of a non existent player i.e. used to record id of player that has not played
    __win_mask = np.full((1, 3),3,np.int8)
    __actions = {1:(0,0), 2:(0,1), 3:(0,2), 4:(1,0), 5:(1,1), 6:(1,2), 7:(2,0), 8:(2,1), 9:(2,2)}
    player_X = 1 # numerical value of player X on the board
    player_O = -1 # numerical value of player O on the board
    empty_cell = 0 # value of a free action space on board
    asStr = True

    #
    # Return game to intial state, where no one has played
    # and the board contains no moves.
    #
    def reset(self):
        self.__board = np.zeros((3, 3),np.int8)
        self.__last_board = np.zeros((3, 3),np.int8)
        self.__game_over = False
        self.__game_drawn = False
        self.__player = TicTacToe.__no_player
        self.__last_player = TicTacToe.__no_player
        
    #
    # Constructor has no arguments as it just sets the game
    # to an intial up-played set-up
    #
    def __init__(self):
        self.__board = np.zeros((3, 3),np.int8)
        self.__last_board = np.zeros((3, 3),np.int8)
        self.__game_over = False
        self.__game_drawn = False
        self.__player = TicTacToe.__no_player
        self.__last_player = TicTacToe.__no_player
    
    #
    # Return a displayable version of the entire game.
    #
    def __str__(self):
        s = ""
        s += "Game Over: " + str(self.__game_over) +"\n"
        s += "Player :" + TicTacToe.__player_to_str(self.__player) + "\n"
        s += "Current Board : \n" + str(self.__board)+ "\n"
        s += "Prev Player :" + TicTacToe.__player_to_str(self.__last_player) + "\n"
        s += "Prev Current Board : \n" + str(self.__last_board)+ "\n"
        s += "State" + str(self.state()) + "\n"
        return s
    
    #
    # return player as string "X" or "O"
    #
    @classmethod
    def __player_to_str(cls,self,player):
        if(player == TicTacToe.player_X): return "X"
        if(player == TicTacToe.player_O): return "O"
        return "?"
        
    #
    # Return the actions as a list of integers.
    #
    @classmethod
    def num_actions(cls):
        return len(TicTacToe.__actions)

    #
    # Return the actions as a list of integers.
    #
    @classmethod
    def actions(cls):
        return list(map(lambda a: int(a), list(TicTacToe.__actions.keys())))

    #
    # Return the board index (i,j) of a given action
    #
    @classmethod
    def board_index(cls,action):
        return TicTacToe.__actions[action]

    #
    # Return rewards as dictionary where key is name of reward
    # and the value is the reward
    #
    @classmethod
    def rewards(cls):
        return TicTacToe.__rewards
    
    #
    # Assume the move has been validated by move method
    # Make a copy of board before move is made and the last player
    #
    def __make_move(self, action, player):
        self.__last_board = np.copy(self.__board)
        self.__last_player = self.__player
        self.__player = player
        self.__board[TicTacToe.board_index(action)] = player
        return
    
    #
    # Has a player already moved using the given action.
    #
    def __valid_move(self,action):
        return self.__board[TicTacToe.board_index(action)] != TicTacToe.empty_cell
    
    #
    # If the proposed action is a valid move and the game is not
    # over. Make the given move (action) on behalf of the given 
    # player and update the game status.
    #
    # return the rawards (Player who took move, Observer)
    #
    def move(self, action, player):
        if(self.game_won()) : return TicTacToe.__bad_move_game_is_over
        if(self.__valid_move(action)): return TicTacToe.__bad_move_action_already_played 
        if(player == self.__player): return TicTacToe.__bad_move_no_consecutive_plays 
        
        self.__make_move(action,player)

        if(self.game_won()):
            self.__game_over = True
            self.__game_drawn = False
            return np.array([TicTacToe.__win,TicTacToe.__loss])
            
        if(not self.moves_left_to_take()):
            self.__game_over = True
            self.__game_drawn = True
            return np.array([TicTacToe.__draw,TicTacToe.__draw])

        return np.array([TicTacToe.__play,0])

    #
    # Return (flattened) Game Ended, Last Player, Last Board, Player, Board
    #
    def detailed_state(self):
        flattened_state = []
        if(self.__game_over):
            flattened_state.append(1)
        else:
            flattened_state.append(0)
        flattened_state.append(self.__last_player)
        flattened_state.append(self.__player)
        for itm in np.reshape(self.__last_board,9).tolist() : flattened_state.append(itm)
        for itm in np.reshape(self.__board,9).tolist() : flattened_state.append(itm)
            
        return flattened_state

    #
    # Show return the current board contents
    #
    def board(self):
        return self.__board
     
    #
    # Any row, column or diagonal with all player X or player O. If a
    # player is given then it answers has that specific player won
    #
    def game_won(self,board=None,plyr=None):
    
        if board is None: 
            bd = self.__board
        else:
            bd = board
        if not plyr is None: bd = (bd==plyr)*1
        
        rows = np.abs(np.sum(bd,axis=1))
        cols = np.abs(np.sum(bd,axis=0))
        diagLR = np.abs(np.sum(bd.diagonal()))
        diagRL = np.abs(np.sum(np.rot90(bd).diagonal()))        
    
        if(np.sum(rows == 3) > 0):
            return True
        if(np.sum(cols == 3) > 0):
            return True
        if((np.mod(diagLR,3)) == 0) and diagLR > 0:
            return True
        if((np.mod(diagRL,3)) == 0) and diagRL > 0:
            return True
        return False

    #
    # Is the game over ?
    #
    def game_over(self):
        return (self.game_won() or not self.moves_left_to_take())
    
    #
    # Return which player goes next given the current player
    #
    @staticmethod
    def next_player(current_player):
        if(current_player == TicTacToe.player_O):
            return  TicTacToe.player_X
        else:
            return  TicTacToe.player_O

    #
    # Are there any remaining moves to be taken >
    #
    def moves_left_to_take(self):
        return (self.__board[np.where(self.__board == 0)]).size > 0
    
    #
    # What moves are valid given for board or if not
    # for the current game board.
    #
    def what_are_valid_moves(self, bd = None):
        if bd == None: bd = self.__board
        vm = np.zeros(TicTacToe.num_actions())
        best_action = None
        for actn in TicTacToe.actions():
            if(bd[TicTacToe.board_index(actn)] == 0):
                vm[int(actn)-1] = True
            else:
                vm[int(actn)-1] = False
        return vm


In [476]:
class PlayTicTacToe:

    #
    # Constructor has no arguments as it just sets the game
    # to an intial up-played set-up
    #
    def __init__(self):
        self.__game = TicTacToe()
        self.__Q = {}
    
    #
    # Return the current game
    #
    def game(self):
        return self.__game
    
    #
    # Set leared state to given QValues.
    #
    def transfer_learning(self,QV):
        self.__Q = QV
        print("Learned Games:" + str(len(self.__Q)))
        
    #
    # The learned Q Values for a given state if they exist
    #
    def Q_Vals_for_state(self,state):
        if(state in self.__Q):
            return(self.__Q[state])
        else:
            return(None)

    #
    # Forget learning
    #
    def forget_learning(self):
        self.__Q = {}

    #
    # Add states to Q Value dictionary if not present
    #
    def add_states_if_missing(self,s1,sp1):
        if s1 not in self.__Q:
            self.__Q[s1] = np.zeros(TicTacToe.num_actions())
        if sp1 not in self.__Q:
            self.__Q[sp1] = np.zeros(TicTacToe.num_actions())

    #
    # Update the Q values for the given player state and
    # the given reward
    #
    def update_Q_Values_for_player(self,mv,s,sp,reward,learning_rate,discount_rate):
        (self.__Q[sp])[mv-1] = (self.__Q[sp])[mv-1] + (learning_rate * reward)
        (self.__Q[s])[mv-1] = learning_rate * (self.__Q[s])[mv-1] + (1-learning_rate) * (discount_rate * np.max(self.__Q[sp]))
        #print("["+ s +"] - > [" + sp +"]")
        #print("["+ str(self.__Q[s]) +"] - > [" + str(self.__Q[sp]) +"]")
        #print(">>")
        return
    
    #
    # Keep Score of players as Q Val Trains.
    #
    @classmethod
    def __init_score(cls):
        score = {}
        score[TicTacToe.player_X]={}
        score[TicTacToe.player_O]={}
        for rn,rv in TicTacToe.rewards().items():
            score[TicTacToe.player_X][rv] = 0
            score[TicTacToe.player_O][rv] = 0
        return score

    @classmethod
    def __keep_score(cls,score,plyr,nxt_plyr,reward):                
        (score[plyr])[reward[0]] += 1
        (score[nxt_plyr])[reward[1]] += 1
        return

    #
    # Return the State, Action Key from the perspective of given player
    #
    @classmethod
    def state(cls,player,board):                
        sa = ""
        sa += str(player)
        for cell in np.reshape(board,9).tolist() : sa+= str(cell)            
        return sa

    #
    # Run simulation to estimate Q values for state, action pairs. Random exploration policy
    # which should be tractable with approx 6K valid board states.
    #
    def train_Q_values(self,num_simulations,canned_moves=None):
        exploration = 1.0
        decay = (1.0/num_simulations)
        learning_rate0 = 0.05
        learning_rate_decay = 0.1
        discount_rate = 0.95
        reward = 0
        sim = 0
        game_step = 0
        plyr = None
        prev_plyr = None
        s1 = None
        sp1 = None
        s2 = None
        sp2 = None
        score = PlayTicTacToe.__init_score()
            
        while(sim < num_simulations):
            self.__game.reset()
            if canned_moves is None:
                plyr = (TicTacToe.player_X,TicTacToe.player_O)[randint(0,1)] # Random player to start
                nxt_plyr = TicTacToe.next_player(plyr)
            mv = None
            while(not self.__game.game_over()):
                
                if canned_moves is None:
                    #random.random() < (exploration-(decay*sim))):
                    mv = self.random_move()
                else:
                    plyr,mv = (canned_moves[sim])[game_step]
                    prev_plyr = TicTacToe.next_player(plyr)
                
                s1 = PlayTicTacToe.state(plyr,self.__game.board())
                s2 = PlayTicTacToe.state(prev_plyr,self.__game.board())
                reward = self.__game.move(mv,plyr)
                sp1 = PlayTicTacToe.state(plyr,self.__game.board())
                sp2 = PlayTicTacToe.state(prev_plyr,self.__game.board())

                learning_rate = learning_rate0 / (1 + (sim * learning_rate_decay))
                
                self.add_states_if_missing(s1,sp1)
                self.add_states_if_missing(s2,sp2)
                
                # Update Q Values for both players based on last play reward.
                self.update_Q_Values_for_player(mv,s1,sp1,reward[0],learning_rate,discount_rate)
                self.update_Q_Values_for_player(mv,s2,sp2,reward[1],learning_rate,discount_rate)
                
                if canned_moves is None:
                    plyr = TicTacToe.next_player(plyr)
                    prev_plyr = plyr
                game_step += 1
            sim += 1
            game_step = 0
            
            PlayTicTacToe.__keep_score(score,plyr,prev_plyr,reward)
            
            if ((sim % 1000) == 0) or (sim == num_simulations) : 
                smX = "Player X : " + str(sim) + " : "
                smO = "Player O : " + str(sim) + " : "
                for rn,rv in TicTacToe.rewards().items():
                    smX += rn+" : "+str(round(((score[TicTacToe.player_X])[rv]/sim)*100,0))+"% "
                    smO += rn+" : "+str(round(((score[TicTacToe.player_O])[rv]/sim)*100,0))+"% "
                print(smX)
                print(smO)
        return self.__Q
    #
    # Return a random action (move) that is still left
    # to make
    #
    def random_move(self):
        valid_moves = []
        random_action = None
        for actn in self.__game.actions():
            if(self.__game.board()[TicTacToe.board_index(actn)] == TicTacToe.empty_cell):
                valid_moves.append(actn)
         
        num_poss_moves = len(valid_moves)
        if(num_poss_moves > 0):
            random_action = valid_moves[randint(0, num_poss_moves-1)]
            return random_action
        else:
            return None
        
    #
    # Given current state and lerned Q Values (if any) suggest
    # the move that is expected to yield the highest reward.
    #
    def informed_move(self,st,rnd):
        # What moves are possible at this stage
        valid_moves = self.__game.what_are_valid_moves()
        
        # Are there any moves ? 
        if(np.sum(valid_moves*np.full(9,1)) == 0):
            return None
    
        best_action = None
        if(not rnd):
            # Is there info learned for this state ?
            informed_actions = self.Q_Vals_for_state(st)
            if not informed_actions is None:
                informed_actions *= valid_moves
                best_action = np.max(informed_actions)
                if(best_action > 0):
                    informed_actions = np.arange(1,TicTacToe.num_actions()+1,1)[np.where(informed_actions == best_action)]
                    best_action = informed_actions[randint(0, informed_actions.size-1)]
                else:
                    best_action = None

        # If we found a good action then return that 
        # else pick a random action
        if best_action == None:
            actions = valid_moves*np.arange(1,TicTacToe.num_actions()+1,1)
            actions = actions[np.where(actions > 0)]
            best_action = actions[randint(0,actions.size-1)]

        return int(best_action)
        
    #
    # Play an automated game between a random player and an
    # informed player. 
    # Return the move sequence for the entire game as s string.
    #
    def play(self):
        self.__game.reset()
        plyr = (TicTacToe.player_X,TicTacToe.player_O)[randint(0,1)] # Chose random player to start
        mv = None
        profile= ""
        while(not self.__game.game_over()):
            st = self.__game.state(True,plyr)
            QV = self.Q_Vals_for_state(st)
            mx = np.max(self.Q_Vals_for_state(st))
            if(plyr == TicTacToe.player_X):
                mv = self.informed_move(st,False) # Informed Player
            else:
                mv = self.informed_move(st,True) # Random Player
            self.__game.move(mv,plyr)
            profile += str(plyr)+":"+str(mv)+"~"
            plyr = TicTacToe.next_player(plyr)
        return profile
    
    #
    # Add the game profile to the given game dictionary and
    # up the count for the number of times that games was played
    #
    @classmethod    
    def record_game_stats(cls,D,profile):
        if profile in D:
            D[profile] += 1
        else:
            D[profile] = 1
        return
    
    def play_many(self,num):
        informed_wins = 0
        random_wins = 0
        draws = 0
        I = {}
        R = {}
        D = {}
        G = {}
        profile = ""
        for x in range(0, num):
            profile = self.play()
            if profile not in G: G[profile]=""
            if self.__game.game_won(self.__game.board(),TicTacToe.player_X):
                informed_wins += 1
                PlayTicTacToe.record_game_stats(I,profile)
            else:
                if self.__game.game_won(self.__game.board(),TicTacToe.player_O):
                    random_wins +=1
                    PlayTicTacToe.record_game_stats(R,profile)
                else: 
                    PlayTicTacToe.record_game_stats(D,profile)
                    draws += 1
            if(x % 100) == 0 : print (str(x))
        print("Informed :" +  str(informed_wins)+" : " + str(round((informed_wins/num)*100,0)))
        print("Random :" +  str(random_wins)+" : " + str(round((random_wins/num)*100,0)))
        print("Draw :" + str(draws)+" : " + str(round((draws/num)*100,0)))
        print("Diff Games :" +  str(len(G)))
        return (I,R,D)
    
    #
    # Convert a game profile string returned from play method
    # into an array that can be passed as a canned-move to
    # training. (Q learn)
    #
    @classmethod    
    def move_str_to_array(cls,moves_as_str):
        mvd = {}
        mvc = 0
        mvs = moves_as_str.split('~')
        for mv in mvs:
            if(len(mv)>0):
                pl,ps = mv.split(":")
                mvd[mvc]=(int(pl),int(ps))
            mvc +=1
        return mvd

    #
    # Convert a dictionary or game profiles returned from play_many
    # to a dictionary of canned moves that can be passed to training (Q Learn)
    #
    @classmethod    
    def moves_to_dict(cls,D):
        MD = {}
        i = 0
        for mvss,cnt in D.items():
            MD[i] = PlayTicTacToe.move_str_to_array(mvss)
            i+=1
        return MD

In [477]:
#QV1=QV
#play.transfer_learning(QV1)

In [478]:
random.seed(42)
np.random.seed(42)
play = PlayTicTacToe()
play.forget_learning()
QV = {}

In [482]:
G1 = {'1:8~-1:1~1:6~-1:3~1:9~-1:2~': 1}
QV = play.train_Q_values(len(G1),PlayTicTacToe.moves_to_dict(G1))

Player X : 1 : Loss : 100.0% Draw : 0.0% Play : 0.0% Win : 0.0% 
Player O : 1 : Loss : 0.0% Draw : 0.0% Play : 0.0% Win : 100.0% 


In [479]:
print(QV)

{}


In [241]:
GI = {}
GR = {}
GD = {}
GI,GR,GD = play.play_many(500)

0
100
200
300
400
Informed :160 : 32.0
Random :157 : 31.0
Draw :183 : 37.0
Diff Games :379


In [270]:
GI = {}
GR = {}
GD = {}
GI,GR,GD = play.play_many(5)
print(GI)
print(GR)
print(GD)


0
Informed :1 : 20.0
Random :3 : 60.0
Draw :1 : 20.0
Diff Games :5
{'1:8~-1:6~1:9~-1:1~1:2~-1:3~1:4~-1:7~1:5~': 1}
{'-1:4~1:8~-1:5~1:1~-1:9~1:2~-1:3~1:7~-1:6~': 1, '-1:6~1:4~-1:7~1:8~-1:5~1:2~-1:9~1:1~-1:3~': 1, '1:8~-1:9~1:3~-1:1~1:4~-1:5~': 1}
{'1:8~-1:9~1:3~-1:7~1:1~-1:4~1:5~-1:2~1:6~': 1}


In [437]:
#GI = {}
#GR = {}
#GD = {}
#for j in range (0,10):
#    GI,GR,GD = play.play_many(50)
#    QV = play.train_Q_values(len(GR),PlayTicTacToe.moves_to_dict(GR))
    #QV = play.train_Q_values(len(GI),PlayTicTacToe.moves_to_dict(GI))
    #QV = play.train_Q_values(len(GD),PlayTicTacToe.moves_to_dict(GD))
print("")

In [ ]:
#
# Run A Single Test
#
test_to_run = "test_one"
suite = unittest.TestSuite()
suite.addTest(testTicTacToe(test_to_run))
runner = unittest.TextTestRunner()
runner.run(suite)

In [ ]:
#
# Run All Tests.
#
tests = testTicTacToe()
suite = unittest.TestLoader().loadTestsFromModule(tests)
unittest.TextTestRunner().run(suite)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [143]:
class NNplayTicTacToe:
    __actor_network_name = "net/actor"
    __critic_network_name = "net/critic"
    __x_units = 21

    def __init__(self):
        return
    
    #
    # Build a four layer fully connected Network.
    # 21 -> relu -> 21 -> relu -> 21 -> relu -> 21 -> relu
    #
    def construct_network(X_state,network_name):
        return

In [480]:
QV = play.train_Q_values(10000)

Player X : 1000 : Loss : 43.0% Draw : 14.0% Play : 0.0% Win : 43.0% 
Player O : 1000 : Loss : 44.0% Draw : 13.0% Play : 0.0% Win : 44.0% 
Player X : 2000 : Loss : 45.0% Draw : 12.0% Play : 0.0% Win : 45.0% 
Player O : 2000 : Loss : 43.0% Draw : 12.0% Play : 0.0% Win : 43.0% 
Player X : 3000 : Loss : 45.0% Draw : 12.0% Play : 0.0% Win : 45.0% 
Player O : 3000 : Loss : 42.0% Draw : 12.0% Play : 0.0% Win : 42.0% 
Player X : 4000 : Loss : 46.0% Draw : 12.0% Play : 0.0% Win : 46.0% 
Player O : 4000 : Loss : 42.0% Draw : 12.0% Play : 0.0% Win : 42.0% 
Player X : 5000 : Loss : 45.0% Draw : 13.0% Play : 0.0% Win : 45.0% 
Player O : 5000 : Loss : 42.0% Draw : 12.0% Play : 0.0% Win : 42.0% 
Player X : 6000 : Loss : 45.0% Draw : 13.0% Play : 0.0% Win : 45.0% 
Player O : 6000 : Loss : 42.0% Draw : 12.0% Play : 0.0% Win : 42.0% 
Player X : 7000 : Loss : 45.0% Draw : 13.0% Play : 0.0% Win : 45.0% 
Player O : 7000 : Loss : 42.0% Draw : 12.0% Play : 0.0% Win : 42.0% 
Player X : 8000 : Loss : 45.0% Dra

In [441]:
def play_ml(playg):
    st = PlayTicTacToe.state(TicTacToe.player_X,playg.game().board())
    QV = playg.Q_Vals_for_state(st)
    mx = np.max(playg.Q_Vals_for_state(st))
    print(playg.game().board())
    print(st)
    print(QV)
    print((QV-mx))
    print((QV==mx)*mx)
    
    #mv = self.informed_move(st,True) # Random Player
    playg.game().move(playg.informed_move(st,False),TicTacToe.player_X)
    print(playg.game().board())

    return

def play_me(playg,mv):
    st = PlayTicTacToe.state(TicTacToe.player_X,playg.game().board())
    QV = playg.Q_Vals_for_state(st)
    mx = np.max(playg.Q_Vals_for_state(st))
    print(playg.game().board())
    playg.game().move(mv,TicTacToe.player_O)
    print(playg.game().board())

    return


In [484]:
play.game().reset()

In [487]:
play_ml(play)

[[-1  0  0]
 [ 0  0  0]
 [ 1  0  0]]
1-100000100
[ 0.          0.04488134  0.0010641   0.0040215   0.01399091  0.0015452
  0.02085563  0.00193042  0.00217061]
[-0.04488134  0.         -0.04381724 -0.04085984 -0.03089043 -0.04333614
 -0.0240257  -0.04295091 -0.04271073]
[ 0.          0.04488134  0.          0.          0.          0.          0.
  0.          0.        ]
[[-1  1  0]
 [ 0  0  0]
 [ 1  0  0]]


In [488]:
play_me(play,9)

[[-1  1  0]
 [ 0  0  0]
 [ 1  0  0]]
[[-1  1  0]
 [ 0  0  0]
 [ 1  0 -1]]


In [483]:
print(play.Q_Vals_for_state("1-10-1001011"))
print(play.Q_Vals_for_state("-1-10-1001011"))

[ 0.          0.          0.          0.          0.          0.00035175
  0.          0.          0.05011146]
[  0.00000000e+00   2.24322345e-01   0.00000000e+00   0.00000000e+00
   1.05884974e-04   0.00000000e+00   1.72226250e-04   0.00000000e+00
   0.00000000e+00]
